In [ ]:
!pip install psutil

In [ ]:
import requests
import json
from datetime import datetime, timezone
import psutil
import platform
from time import sleep

In [ ]:
# SET VARIABLES
istsos_url = 'http://istsos.org/istsos'
istsos_service = 'maxdemo'
procedure_name = 'LAPTOP'

In [ ]:
uname = platform.uname()
print(f"System:           {uname.system}")
print(f"Node Name:        {uname.node}")
print(f"Version:          {uname.version}")
print(f"Processor:        {uname.processor}")
print(f"Total CPU Usage:  {psutil.cpu_percent()}%")
print(f"Percentage memory:{psutil.virtual_memory().percent}%")
print(f"Python version:   {platform.python_version()}")

In [ ]:
procedure_dict = {
    'system_id': procedure_name,
    'system': procedure_name,
    'description': 'Laptop cpu and memory monitoring',
    'keywords': 'cpu,sensor,load,memory',
    'identification': [
        {
            'name': 'uniqueID',
            'definition': 'urn:ogc:def:identifier:OGC:uniqueID',
            'value': "urn:ogc:def:procedure:x-istsos:1.0:{}".format(procedure_name)
        }
    ],
    'classification': [
        {
            'name': 'System Type',
            'definition': 'urn:ogc:def:classifier:x-istsos:1.0:systemType',
            'value': 'insitu-fixed-point'
        },
        {
            'name': 'Sensor Type',
            'definition': 'urn:ogc:def:classifier:x-istsos:1.0:sensorType',
            'value': 'laptop'
        }
    ],
    'characteristics': '',
    'contacts': [],
    'documentation': [],
    'capabilities': [],
    'location': {
        'type': 'Feature',
        'geometry': {
            'type': 'Point',
            'coordinates': ['8', '45', '200']
        },
        'crs': {
            'type': 'name',
            'properties': {'name': '4326'}
        },
        'properties': {
            'name': 'TREVANO'
        }
    },
    'interfaces': '',
    'inputs': [],
    'outputs': [
        {
            'name': 'Time',
            'definition': 'urn:ogc:def:parameter:x-istsos:1.0:time:iso8601',
            'uom': 'iso8601', 'description': ''
        },
        {
            'name': 'cpu-load',
            'definition': 'urn:ogc:def:parameter:x-istsos:1.0:pc:cpu:load',
            'uom': '%',
            'description': 'Instant load of the CPU'
        }, {
            'name': 'cpu-temperature',
            'definition': 'urn:ogc:def:parameter:x-istsos:1.0:pc:memory:load',
            'uom': '%',
            'description': 'Instant load of the memory'
        }
    ],
    'history': [],
    'mqtt': None
}

In [ ]:
req = requests.post(
    f'{istsos_url}/wa/istsos/services/{istsos_service}/procedures',
    data=json.dumps(procedure_dict)
)
if req.json()['success']:
    print('SUCCESS SENSOR REGISTERD!!!')
else:
    print('ERROR SENSOR NOT REGISTERED')

In [ ]:
assigned_id = '75493eb45f1211eaaf0608002747b72e'

In [ ]:
count = 120
for _ in range(count):
    data_post = '{};{},{}'.format(
        assigned_id,
        datetime.now(timezone.utc).isoformat(),
        '{},{}'.format(
            psutil.cpu_percent(),
            psutil.virtual_memory().percent
        )
    )
    print(data_post)
    req = requests.post(
        '{}/wa/istsos/services/{}/operations/fastinsert'.format(
            istsos_url,
            istsos_service,
        ),
        data=data_post
    )
    if 'true' in req.text:
        print('SUCCESS DATA SENT!!!')
    else:
        print('ERROR DATA NOT SENT')
    sleep(3.0)